# Project prototype 2

### Your name
Write the name(s) of the author(s) of this assignment below:
* Brian Westerweel
* Christoph Schmidl
* Gijs van der Meijde
* Jeffrey Luppus

In [5]:
# System
from os.path import join, basename, dirname, exists  
import os  
from glob import glob 
import random 
from tqdm import tqdm

# Computational
import pandas as pd  
import numpy as np  
from matplotlib import pyplot as plt  

# Deep learning
import keras

# Other
from scipy.ndimage import imread  
from IPython.display import clear_output
from sklearn.metrics import precision_recall_curve

Using TensorFlow backend.


## Data Download

## If the Jupter server runs on Cartesius

In [6]:
train_dir = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/train_images'
validation_dir = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/validation_images'
test_dir = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/test_images'
train_points = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/training_annotations.csv'
data_dir = '/home/ruc0030/proj_1/project_data'
train_masks_dir = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/mask_images'

In [77]:
#we have THE BEST patch generator, its true, its great.

#This function generates 1 random patch from patch_amount random images.
def generate_patches(img_dir, msk_dir, patch_amount, width, height):
    print("Generating {} batches of {}x{} from {}/*.png".format(patch_amount,width,height,img_dir))
    patches = []
    all_img_paths = os.listdir(img_dir)
    print("{} images found.".format(len(all_img_paths)))
    img_paths = random.sample(all_img_paths, patch_amount)
    
    for img_path in img_paths:
        img = imread(join(img_dir,img_path))
        msk = imread(join(msk_dir,img_path.split(".")[0],"_mask.png"))
        print("img {} dementions: {}".format(img_path,img.shape))
        patches.append(generate_patch(img,msk,width,height))
        
    return patches

#This function generates a single random patch from an image.
def generate_patch(img, msk, width, height):
    img = np.pad(img, (0 if height < img.shape[1] else height-img.shape[1] , 0 if width < img.shape[0] else width-img.shape[0]), mode='constant')
    img = np.pad(msk, (0 if height < img.shape[1] else height-img.shape[1] , 0 if width < img.shape[0] else width-img.shape[0]), mode='constant')
    x = random.randint(0,img.shape[0]-width+1);
    x1 = x + width
    y = random.randint(0,img.shape[1]-height+1);
    y1 = y + height
    print("patch: {}x{}".format(x,y))
    return (img[x:x1,y:y1,:], msk[x:x1,y:y1,:])

In [78]:
generate_patches(train_dir, train_masks_dir, 20, 512, 512)

Generating 20 batches of 512x512 from /projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/train_images/*.png
413 images found.
img 35_CD8_ROI_9.png dementions: (438, 494, 3)
patch: 10x51
img T2C03L1A2B1S11R01_ROI_6.png dementions: (410, 569, 3)
patch: 1x74


/hpc/sw/python-3.5.2/lib/python3.5/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  if sys.path[0] == '':


img 95-22625-I-4_CD8_ROI_6.png dementions: (1765, 2628, 3)
patch: 173x315
img 3_CD8_ROI_9.png dementions: (826, 727, 3)
patch: 158x200
img T3C03L1A2B1S11R01_ROI_2.png dementions: (5583, 6635, 3)
patch: 4951x3952
img T2C03L1A2B1S11R01_ROI_2.png dementions: (1713, 1959, 3)
patch: 55x488
img T10-4360_I_AG_CD3_ROI_2.png dementions: (844, 1005, 3)
patch: 102x227
img 95-30827-5_CD3_ROI_12.png dementions: (1113, 1969, 3)
patch: 134x955
img 3_CD8_ROI_1.png dementions: (3999, 6650, 3)
patch: 101x4070
img T17-071841_III6_CD8_ROI_8.png dementions: (1427, 1846, 3)
patch: 797x1001
img 31_CD3_ROI_11.png dementions: (747, 1014, 3)
patch: 227x141
img 95-00909-II-L1_CD8_ROI_13.png dementions: (895, 1247, 3)
patch: 364x717
img T17-071825_III4_CD3_ROI_3.png dementions: (4003, 3650, 3)
patch: 1935x2369
img 3_CD8_ROI_12.png dementions: (558, 706, 3)
patch: 27x91
img 95-22625-I-4_CD8_ROI_3.png dementions: (2729, 4247, 3)
patch: 118x3675
img 1080_CD3_ROI_1.png dementions: (1489, 1851, 3)
patch: 640x430
img 1

[array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ..., 
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ..., 
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ..., 
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ..., 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ..., 
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 